In [ ]:
# If use jupyter notebook, cd to the path of the folder all-rnr-annotated-threads



D:\Projects\School\CSI4900\Fake-News-Detection\raw-data\pheme\all-rnr-annotated-threads


d:\Projects\School\CSI4900\Fake-News-Detection\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install pyarrow


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tarfile
tar = tarfile.open("/content/drive/MyDrive/CSI4900/Datasets/ForTesting/PHEME_veracity.tar.bz2")
tar.extractall("/content/drive/MyDrive/CSI4900/Datasets/ForTesting/Pheme")
tar.close()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os

folds = [
    'charliehebdo-all-rnr-threads',
    'ottawashooting-all-rnr-threads',
    'ebola-essien-all-rnr-threads',
    'prince-toronto-all-rnr-threads',
    'ferguson-all-rnr-threads',
    'putinmissing-all-rnr-threads',
    'germanwings-crash-all-rnr-threads',
    'gurlitt-all-rnr-threads',
    'sydneysiege-all-rnr-threads'
]


texts = []
fav_counts = []
retweet_counts = []
date = []

username = []
account_date = []
protected = []
verified = []
followers = []
followings = []
tweets_count = []

hashtag = []
url = []

events = []
y = []

def convert_annotations_data(annotation, string = True):
    label = None
    if 'is_rumour' in annotation.keys() and annotation['is_rumour'] == 'nonrumour':
        label = 'true'
    elif 'misinformation' in annotation.keys() and 'true'in annotation.keys():
        if int(annotation['misinformation'])==0 and int(annotation['true'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==0 and int(annotation['true'])==1 :
            if string:
                label = "true"
            else:
                label = 1
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==0 :
            if string:
                label = "false"
            else:
                label = 0
        elif int(annotation['misinformation'])==1 and int(annotation['true'])==1:
            print ("OMG! They both are 1!")
            print(annotation['misinformation'])
            print(annotation['true'])
            label = None

    elif 'misinformation' in annotation.keys() and 'true' not in annotation.keys():
        # all instances have misinfo label but don't have true label
        if int(annotation['misinformation'])==0:
            if string:
                label = "unverified"
            else:
                label = 2
        elif int(annotation['misinformation'])==1:
            if string:
                label = "false"
            else:
                label = 0

    elif 'true' in annotation.keys() and 'misinformation' not in annotation.keys():
        print ('Has true not misinformation')
    else:
        print('No annotations')

    return label

In [ ]:
for f in folds:
  paths = [
     os.path.join('/content/drive/MyDrive/CSI4900/Datasets/ForTesting/Pheme/all-rnr-annotated-threads', f, 'rumours'),
     os.path.join('/content/drive/MyDrive/CSI4900/Datasets/ForTesting/Pheme/all-rnr-annotated-threads', f, 'non-rumours')
  ]
  for path in paths:
    for dir1 in os.listdir(path):
          if '_' not in dir1:
            path_target  = os.path.join(path,dir1,'annotation.json')
            try:
                file = open(path_target)
                data = json.load(file)
                target = convert_annotations_data(data)
                y.append(target)
            except:
                 print(path_target + " not found!")

            path2 = os.path.join(path, dir1,'source-tweets')
            for dir2 in os.listdir(path2):
              if '_' not in dir2:
                path3  = os.path.join(path2,dir2)
                file = open(path3)
                data = json.load(file)

                #tweet features
                text = data['text']
                tweet_date = data['created_at']
                fav = data['favorite_count']
                retw = data['retweet_count']

                #user features
                usernames = data['user']['screen_name']
                account_creation = data['user']['created_at']
                is_protected = data['user']['protected']
                is_verified = data['user']['verified']
                no_followers = data['user']['followers_count']
                no_followings = data['user']['friends_count']
                no_tweets = data['user']['statuses_count']

                #entities
                no_hashtags = len(data['entities']['hashtags'])
                has_url = data['entities']['urls']


                texts.append(text)
                date.append(tweet_date)
                fav_counts.append(fav)
                retweet_counts.append(retw)

                username.append(usernames)
                account_date.append(account_creation)
                protected.append(is_protected)
                verified.append(is_verified)
                followers.append(no_followers)
                followings.append(no_followings)
                tweets_count.append(no_tweets)


                hashtag.append(no_hashtags)
                url.append(has_url)

                events.append(f)

/content/drive/MyDrive/CSI4900/Datasets/ForTesting/Pheme/all-rnr-annotated-threads/charliehebdo-all-rnr-threads/non-rumours/553566279953842176/annotation.json not found!


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/CSI4900/Datasets/ForTesting/Pheme/all-rnr-annotated-threads/charliehebdo-all-rnr-threads/non-rumours/553566279953842176/source-tweets'

In [ ]:
df = pd.DataFrame([texts,date,fav_counts,retweet_counts,username,account_date,followers,followings,tweets_count,protected,verified,hashtag,url,events,y],['text','date','fav_count','retweet_count','username','account_date','followers','followings','tweet_count','protected','verified','no_hashtags','urls','event','target']).transpose()
df = df.infer_objects()
df.dropna(inplace=True)

In [ ]:
df.ino()


In [ ]:
len(df.index)

6424

In [ ]:
# original dataframe
df.head(5)

,text,date,fav_count,retweet_count,username,account_date,followers,followings,tweet_count,protected,verified,no_hashtags,urls,event,target
0,"Breaking: At least 10 dead, 5 injured after tO...",Wed Jan 07 11:06:08 +0000 2015,14,159,H_E_Samuel,Tue Oct 04 09:36:17 +0000 2011,1628,246,1901,False,False,0,[],charliehebdo-all-rnr-threads,true
1,France: 10 people dead after shooting at HQ of...,Wed Jan 07 11:07:51 +0000 2015,38,486,euronews,Wed Mar 18 12:57:11 +0000 2009,129573,337,48440,False,True,1,[],charliehebdo-all-rnr-threads,true
2,Ten killed in shooting at headquarters of Fren...,Wed Jan 07 11:08:09 +0000 2015,15,127,Channel4News,Mon Apr 28 14:29:37 +0000 2008,529882,3051,66682,False,True,1,[],charliehebdo-all-rnr-threads,true
3,BREAKING: 10 dead in shooting at headquarters ...,Wed Jan 07 11:09:50 +0000 2015,15,105,AlArabiya_Eng,Sat Feb 28 08:31:32 +0000 2009,499741,31,100466,False,True,2,[],charliehebdo-all-rnr-threads,true
4,Reuters: 10 people shot dead at headquarters o...,Wed Jan 07 11:11:16 +0000 2015,32,412,SkyNewsBreak,Wed Nov 04 11:25:45 +0000 2009,1377384,6,27228,False,True,1,[],charliehebdo-all-rnr-threads,true


In [ ]:
df.tail(5)

,text,date,fav_count,retweet_count,username,account_date,followers,followings,tweet_count,protected,verified,no_hashtags,urls,event,target
6420,Sydney siege ends as police storm Lindt Cafe a...,Mon Dec 15 15:55:59 +0000 2014,18,100,guardian,Thu Nov 05 23:49:19 +0000 2009,2997545,1081,102254,False,True,0,"[{'url': 'http://t.co/ek1EyHbJxA', 'indices': ...",sydneysiege-all-rnr-threads,true
6421,"Breaking News: #SydneySiege is over, according...",Mon Dec 15 15:56:41 +0000 2014,315,489,FoxNews,Sat Mar 17 19:01:26 +0000 2007,4636658,382,188589,False,True,1,"[{'url': 'http://t.co/7p9QLG6mn4', 'indices': ...",sydneysiege-all-rnr-threads,true
6422,Watch gunfire erupt and hostages flee chocolat...,Mon Dec 15 15:58:05 +0000 2014,91,181,NBCNews,Tue Mar 18 23:19:17 +0000 2008,1955488,2894,44356,False,True,1,"[{'url': 'http://t.co/Ju5h01y3R4', 'indices': ...",sydneysiege-all-rnr-threads,true
6423,Authorities have confirmed that #sydneysiege i...,Mon Dec 15 15:58:27 +0000 2014,35,117,YahooNewsUK,Fri Feb 13 15:44:54 +0000 2009,36137,926,14145,False,True,1,"[{'url': 'http://t.co/IrteddYrbH', 'indices': ...",sydneysiege-all-rnr-threads,true
6424,WATCH: The dramatic moment tactical teams stor...,Mon Dec 15 15:58:44 +0000 2014,93,117,PzFeed,Wed May 04 06:32:13 +0000 2011,201981,3540,21790,False,False,0,"[{'url': 'https://t.co/v8UYPgnyog', 'indices':...",sydneysiege-all-rnr-threads,true


In [ ]:
df['target'] = df['target'].replace(['unverified'], 'false')
array = ['false','true']
df_transform = pd.DataFrame(array,columns=['status'])
status_dict = df_transform['status'].unique().tolist()
df['target']=df['target'].apply(lambda x : status_dict.index(x))

df.head(5)

,text,date,fav_count,retweet_count,username,account_date,followers,followings,tweet_count,protected,verified,no_hashtags,urls,event,target
0,"Breaking: At least 10 dead, 5 injured after tO...",Wed Jan 07 11:06:08 +0000 2015,14,159,H_E_Samuel,Tue Oct 04 09:36:17 +0000 2011,1628,246,1901,False,False,0,[],charliehebdo-all-rnr-threads,1
1,France: 10 people dead after shooting at HQ of...,Wed Jan 07 11:07:51 +0000 2015,38,486,euronews,Wed Mar 18 12:57:11 +0000 2009,129573,337,48440,False,True,1,[],charliehebdo-all-rnr-threads,1
2,Ten killed in shooting at headquarters of Fren...,Wed Jan 07 11:08:09 +0000 2015,15,127,Channel4News,Mon Apr 28 14:29:37 +0000 2008,529882,3051,66682,False,True,1,[],charliehebdo-all-rnr-threads,1
3,BREAKING: 10 dead in shooting at headquarters ...,Wed Jan 07 11:09:50 +0000 2015,15,105,AlArabiya_Eng,Sat Feb 28 08:31:32 +0000 2009,499741,31,100466,False,True,2,[],charliehebdo-all-rnr-threads,1
4,Reuters: 10 people shot dead at headquarters o...,Wed Jan 07 11:11:16 +0000 2015,32,412,SkyNewsBreak,Wed Nov 04 11:25:45 +0000 2009,1377384,6,27228,False,True,1,[],charliehebdo-all-rnr-threads,1


In [ ]:
df = df.rename(columns={'target':'label', 'username': 'author', 'no_hashtags': 'hashtags'})
df.head(5)

,text,date,fav_count,retweet_count,author,account_date,followers,followings,tweet_count,protected,verified,hashtags,urls,event,label
0,"Breaking: At least 10 dead, 5 injured after tO...",Wed Jan 07 11:06:08 +0000 2015,14,159,H_E_Samuel,Tue Oct 04 09:36:17 +0000 2011,1628,246,1901,False,False,0,[],charliehebdo-all-rnr-threads,1
1,France: 10 people dead after shooting at HQ of...,Wed Jan 07 11:07:51 +0000 2015,38,486,euronews,Wed Mar 18 12:57:11 +0000 2009,129573,337,48440,False,True,1,[],charliehebdo-all-rnr-threads,1
2,Ten killed in shooting at headquarters of Fren...,Wed Jan 07 11:08:09 +0000 2015,15,127,Channel4News,Mon Apr 28 14:29:37 +0000 2008,529882,3051,66682,False,True,1,[],charliehebdo-all-rnr-threads,1
3,BREAKING: 10 dead in shooting at headquarters ...,Wed Jan 07 11:09:50 +0000 2015,15,105,AlArabiya_Eng,Sat Feb 28 08:31:32 +0000 2009,499741,31,100466,False,True,2,[],charliehebdo-all-rnr-threads,1
4,Reuters: 10 people shot dead at headquarters o...,Wed Jan 07 11:11:16 +0000 2015,32,412,SkyNewsBreak,Wed Nov 04 11:25:45 +0000 2009,1377384,6,27228,False,True,1,[],charliehebdo-all-rnr-threads,1


In [ ]:
# Order the columns in the dataframe to follow the following format
# text, label, author, metadata (where metadata is a key-value pair of everything else)
df = df[['text', 'label', 'author', 'date', 'fav_count', 'retweet_count', 'account_date', 'followers', 'followings', 'tweet_count', 'protected', 'verified', 'hashtags', 'urls', 'event']]
# metadata
df1 = df[['text', 'label', 'author']].assign(metadata=df.iloc[:,3:].agg(dict,1))
df1.head(5)

,text,label,author,metadata
0,"Breaking: At least 10 dead, 5 injured after tO...",1,H_E_Samuel,"{'date': 'Wed Jan 07 11:06:08 +0000 2015', 'fa..."
1,France: 10 people dead after shooting at HQ of...,1,euronews,"{'date': 'Wed Jan 07 11:07:51 +0000 2015', 'fa..."
2,Ten killed in shooting at headquarters of Fren...,1,Channel4News,"{'date': 'Wed Jan 07 11:08:09 +0000 2015', 'fa..."
3,BREAKING: 10 dead in shooting at headquarters ...,1,AlArabiya_Eng,"{'date': 'Wed Jan 07 11:09:50 +0000 2015', 'fa..."
4,Reuters: 10 people shot dead at headquarters o...,1,SkyNewsBreak,"{'date': 'Wed Jan 07 11:11:16 +0000 2015', 'fa..."


In [ ]:
# change false to 1 and true to 0
df1['label'] = df1['label'].apply(lambda x : 1 if x==0 else 0)
df1.head(5)

,text,label,author,metadata
0,"Breaking: At least 10 dead, 5 injured after tO...",0,H_E_Samuel,"{'date': 'Wed Jan 07 11:06:08 +0000 2015', 'fa..."
1,France: 10 people dead after shooting at HQ of...,0,euronews,"{'date': 'Wed Jan 07 11:07:51 +0000 2015', 'fa..."
2,Ten killed in shooting at headquarters of Fren...,0,Channel4News,"{'date': 'Wed Jan 07 11:08:09 +0000 2015', 'fa..."
3,BREAKING: 10 dead in shooting at headquarters ...,0,AlArabiya_Eng,"{'date': 'Wed Jan 07 11:09:50 +0000 2015', 'fa..."
4,Reuters: 10 people shot dead at headquarters o...,0,SkyNewsBreak,"{'date': 'Wed Jan 07 11:11:16 +0000 2015', 'fa..."


In [ ]:
df1.reset_index(drop=True, inplace=True)
df1.to_feather('pheme.feather')

In [ ]:
import pandas as pd

In [ ]:
df_pheme = pd.read_feather("/content/drive/MyDrive/CSI4900/Datasets/ForTesting/PhemeComplete/pheme.feather")
df_pheme

,text,label,author,metadata
0,"Breaking: At least 10 dead, 5 injured after tO...",0,H_E_Samuel,{'account_date': 'Tue Oct 04 09:36:17 +0000 20...
1,France: 10 people dead after shooting at HQ of...,0,euronews,{'account_date': 'Wed Mar 18 12:57:11 +0000 20...
2,Ten killed in shooting at headquarters of Fren...,0,Channel4News,{'account_date': 'Mon Apr 28 14:29:37 +0000 20...
3,BREAKING: 10 dead in shooting at headquarters ...,0,AlArabiya_Eng,{'account_date': 'Sat Feb 28 08:31:32 +0000 20...
4,Reuters: 10 people shot dead at headquarters o...,0,SkyNewsBreak,{'account_date': 'Wed Nov 04 11:25:45 +0000 20...
...,...,...,...,...
6419,Sydney siege ends as police storm Lindt Cafe a...,0,guardian,{'account_date': 'Thu Nov 05 23:49:19 +0000 20...
6420,"Breaking News: #SydneySiege is over, according...",0,FoxNews,{'account_date': 'Sat Mar 17 19:01:26 +0000 20...
6421,Watch gunfire erupt and hostages flee chocolat...,0,NBCNews,{'account_date': 'Tue Mar 18 23:19:17 +0000 20...
6422,Authorities have confirmed that #sydneysiege i...,0,YahooNewsUK,{'account_date': 'Fri Feb 13 15:44:54 +0000 20...


In [ ]:
df_pheme = df_pheme.dropna()
len(df_pheme)

6424

In [ ]:
df_pheme['label'].value_counts()

label
0    5089
1    1335
Name: count, dtype: int64

In [ ]:
df_pheme['domain'] = 'POLITICS'
df_pheme

,text,label,author,metadata,domain
0,"Breaking: At least 10 dead, 5 injured after tO...",0,H_E_Samuel,{'account_date': 'Tue Oct 04 09:36:17 +0000 20...,POLITICS
1,France: 10 people dead after shooting at HQ of...,0,euronews,{'account_date': 'Wed Mar 18 12:57:11 +0000 20...,POLITICS
2,Ten killed in shooting at headquarters of Fren...,0,Channel4News,{'account_date': 'Mon Apr 28 14:29:37 +0000 20...,POLITICS
3,BREAKING: 10 dead in shooting at headquarters ...,0,AlArabiya_Eng,{'account_date': 'Sat Feb 28 08:31:32 +0000 20...,POLITICS
4,Reuters: 10 people shot dead at headquarters o...,0,SkyNewsBreak,{'account_date': 'Wed Nov 04 11:25:45 +0000 20...,POLITICS
...,...,...,...,...,...
6419,Sydney siege ends as police storm Lindt Cafe a...,0,guardian,{'account_date': 'Thu Nov 05 23:49:19 +0000 20...,POLITICS
6420,"Breaking News: #SydneySiege is over, according...",0,FoxNews,{'account_date': 'Sat Mar 17 19:01:26 +0000 20...,POLITICS
6421,Watch gunfire erupt and hostages flee chocolat...,0,NBCNews,{'account_date': 'Tue Mar 18 23:19:17 +0000 20...,POLITICS
6422,Authorities have confirmed that #sydneysiege i...,0,YahooNewsUK,{'account_date': 'Fri Feb 13 15:44:54 +0000 20...,POLITICS


In [ ]:
df_pheme.to_csv("/content/drive/MyDrive/CSI4900/Datasets/ForTesting/PhemeComplete/pheme.csv")

Other

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
testing_pt1 = pd.read_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/domain_classifier_results1_new.csv")
testing_pt2 = pd.read_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/classifier_test_results_2.csv")
testing_pt3_pheme = pd.read_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/pheme_test_results.csv")

In [ ]:
len(testing_pt2)

In [ ]:
all_testing = pd.concat([testing_pt1, testing_pt2, testing_pt3_pheme])
display(all_testing)

,Unnamed: 0.1,index,Unnamed: 0,id,text,expected_domain,predicted_domain
0,0.0,0.0,0,0,Does This Video Show a Thwarted Mothers Day Ro...,CRIME,SOCIAL
1,1.0,1.0,1,1,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL
2,2.0,2.0,2,2,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL
3,3.0,3.0,3,3,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL
4,4.0,4.0,4,4,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL
...,...,...,...,...,...,...,...
6419,NaN,NaN,6419,6419,Sydney siege ends as police storm Lindt Cafe a...,POLITICS,HEALTH
6420,NaN,NaN,6420,6420,"Breaking News: #SydneySiege is over, according...",POLITICS,HEALTH
6421,NaN,NaN,6421,6421,Watch gunfire erupt and hostages flee chocolat...,POLITICS,HEALTH
6422,NaN,NaN,6422,6422,Authorities have confirmed that #sydneysiege i...,POLITICS,HEALTH


In [ ]:
all_testing.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/classifier_output_combined.csv")

In [ ]:
all_testing = all_testing.drop_duplicates(subset=['text'])
display(all_testing)

,Unnamed: 0.1,index,Unnamed: 0,id,text,expected_domain,predicted_domain
0,0.0,0.0,0,0,Does This Video Show a Thwarted Mothers Day Ro...,CRIME,SOCIAL
1,1.0,1.0,1,1,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL
3,3.0,3.0,3,3,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL
5,5.0,5.0,5,5,Colored Stickers Target Homes of Dog Owners?,CRIME,SOCIAL
15,15.0,15.0,15,15,"Crime in Sweden, Part III: Does Sweden Have No...",CRIME,SOCIAL
...,...,...,...,...,...,...,...
6419,NaN,NaN,6419,6419,Sydney siege ends as police storm Lindt Cafe a...,POLITICS,HEALTH
6420,NaN,NaN,6420,6420,"Breaking News: #SydneySiege is over, according...",POLITICS,HEALTH
6421,NaN,NaN,6421,6421,Watch gunfire erupt and hostages flee chocolat...,POLITICS,HEALTH
6422,NaN,NaN,6422,6422,Authorities have confirmed that #sydneysiege i...,POLITICS,HEALTH


In [ ]:
all_testing.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/classifier_output_combined_nodups.csv")

In [ ]:
og_pt1 = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/testing_all1.csv")
og_pt2 = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/testing_all2.csv")
og_pt3_pheme = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ForTesting/PhemeComplete/pheme.csv")

In [ ]:
og_pt3_pheme.to_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/politics_pheme.csv")

In [ ]:
# og_pt1 = og_pt1.dropna(subset = ['text'])
# og_pt2 = og_pt2.dropna(subset = ['text'])
# og_pt3_pheme = og_pt3_pheme.dropna(subset = ['text'])

In [ ]:
all_testing_og = pd.concat([og_pt1, og_pt2, og_pt3_pheme])
all_testing_og = all_testing_og.drop_duplicates(subset=['text'])
display(all_testing_og)

,Unnamed: 0,text,label,metadata,domain,author
0,0,Does This Video Show a Thwarted Mothers Day Ro...,0,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN
1,1,Should You Call #77 or 112 to Contact the Poli...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN
3,3,Was a Child Named Chloe Jones Abducted by a Cr...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN
5,5,Colored Stickers Target Homes of Dog Owners?,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN
15,15,"Crime in Sweden, Part III: Does Sweden Have No...",1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN
...,...,...,...,...,...,...
6419,6419,Sydney siege ends as police storm Lindt Cafe a...,0,{'account_date': 'Thu Nov 05 23:49:19 +0000 20...,POLITICS,guardian
6420,6420,"Breaking News: #SydneySiege is over, according...",0,{'account_date': 'Sat Mar 17 19:01:26 +0000 20...,POLITICS,FoxNews
6421,6421,Watch gunfire erupt and hostages flee chocolat...,0,{'account_date': 'Tue Mar 18 23:19:17 +0000 20...,POLITICS,NBCNews
6422,6422,Authorities have confirmed that #sydneysiege i...,0,{'account_date': 'Fri Feb 13 15:44:54 +0000 20...,POLITICS,YahooNewsUK


In [ ]:
all_testing['label'] = all_testing_og['label']
all_testing

,Unnamed: 0.1,index,Unnamed: 0,id,text,expected_domain,predicted_domain,label
0,0.0,0.0,0,0,Does This Video Show a Thwarted Mothers Day Ro...,CRIME,SOCIAL,0
1,1.0,1.0,1,1,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL,1
3,3.0,3.0,3,3,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL,1
5,5.0,5.0,5,5,Colored Stickers Target Homes of Dog Owners?,CRIME,SOCIAL,1
15,15.0,15.0,15,15,"Crime in Sweden, Part III: Does Sweden Have No...",CRIME,SOCIAL,1
...,...,...,...,...,...,...,...,...
6419,NaN,NaN,6419,6419,Sydney siege ends as police storm Lindt Cafe a...,POLITICS,HEALTH,0
6420,NaN,NaN,6420,6420,"Breaking News: #SydneySiege is over, according...",POLITICS,HEALTH,0
6421,NaN,NaN,6421,6421,Watch gunfire erupt and hostages flee chocolat...,POLITICS,HEALTH,0
6422,NaN,NaN,6422,6422,Authorities have confirmed that #sydneysiege i...,POLITICS,HEALTH,0


In [ ]:
all_testing = all_testing.drop(columns=['Unnamed: 0.1', 'index', 'Unnamed: 0'])
all_testing

,id,text,expected_domain,predicted_domain,label
0,0,Does This Video Show a Thwarted Mothers Day Ro...,CRIME,SOCIAL,0
1,1,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL,1
3,3,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL,1
5,5,Colored Stickers Target Homes of Dog Owners?,CRIME,SOCIAL,1
15,15,"Crime in Sweden, Part III: Does Sweden Have No...",CRIME,SOCIAL,1
...,...,...,...,...,...
6419,6419,Sydney siege ends as police storm Lindt Cafe a...,POLITICS,HEALTH,0
6420,6420,"Breaking News: #SydneySiege is over, according...",POLITICS,HEALTH,0
6421,6421,Watch gunfire erupt and hostages flee chocolat...,POLITICS,HEALTH,0
6422,6422,Authorities have confirmed that #sydneysiege i...,POLITICS,HEALTH,0


In [ ]:
all_testing_politics = all_testing[all_testing["predicted_domain"] == 'POLITICS']
all_testing_politics

,id,text,expected_domain,predicted_domain,label
130,130,Did BuzzFeed Advocate for Taking Away White Pe...,CRIME,POLITICS,1
818,818,"Donald Trump stopped in Hawaii, which is total...",SCIENCE,POLITICS,1
819,819,Who knew that Rick Perry would come up with an...,SCIENCE,POLITICS,1
820,820,Like increasing numbers of heartless people on...,SCIENCE,POLITICS,1
821,821,"Let s just put it this way, if world leaders r...",SCIENCE,POLITICS,1
...,...,...,...,...,...
6365,6365,If Sydney had been Texas 10 customers would ha...,POLITICS,POLITICS,0
6366,6366,"Sydney siege on all day, but we haven't seen a...",POLITICS,POLITICS,0
6371,6371,"A solitary lunatic, with a history of trauma, ...",POLITICS,POLITICS,0
6373,6373,US soccer has a new First Couple: USWNT's Sydn...,POLITICS,POLITICS,0


In [ ]:
all_testing_politics.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/FurtherTesting_Datasets/further_testing_politics.csv")

In [ ]:
all_testing_crime = all_testing[all_testing["predicted_domain"] == 'CRIME']
all_testing_crime.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/FurtherTesting_Datasets/further_testing_crime.csv")
all_testing_crime

,id,text,expected_domain,predicted_domain,label
1866,1866,"GAITHERSBURG, Md. and GUANGZHOU, China , Jan. ...",HEALTH,CRIME,1
1878,1878,Democrats say fitting submarine with nuclear w...,HEALTH,CRIME,1
2041,2041,Davao City Mayor Sara Duterte on Monday announ...,HEALTH,CRIME,1
2053,2053,In a bid to avert the possible outbreak of Las...,HEALTH,CRIME,1
2080,2080,"A slew of economic reports, such as Retails Sa...",HEALTH,CRIME,1
2095,2095,"As we have said, the silver ray is the feminin...",HEALTH,CRIME,0
2170,2170,"Thus, it is very probable that potential SARS-...",HEALTH,CRIME,0
2227,2227,"TOMI Environmental Solutions, Inc. (TOMI) (OT...",HEALTH,CRIME,0
2854,2854,"TOMI Environmental Solutions, Inc. (TOMI) (OT...",HEALTH,CRIME,0
3040,3040,"In thousands of reportage about coronavirus, t...",HEALTH,CRIME,0


In [ ]:
all_testing_social = all_testing[all_testing["predicted_domain"] == 'SOCIAL']
all_testing_social.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/FurtherTesting_Datasets/further_testing_social.csv")
all_testing_social

,id,text,expected_domain,predicted_domain,label
0,0,Does This Video Show a Thwarted Mothers Day Ro...,CRIME,SOCIAL,0
1,1,Should You Call #77 or 112 to Contact the Poli...,CRIME,SOCIAL,1
3,3,Was a Child Named Chloe Jones Abducted by a Cr...,CRIME,SOCIAL,1
5,5,Colored Stickers Target Homes of Dog Owners?,CRIME,SOCIAL,1
15,15,"Crime in Sweden, Part III: Does Sweden Have No...",CRIME,SOCIAL,1
...,...,...,...,...,...
6352,6352,So #sydneysiege gunman turns out to be a self ...,POLITICS,SOCIAL,0
6400,6400,New South Wales Police confirms the siege in a...,POLITICS,SOCIAL,0
6408,6408,#BREAKING: Australian police confirm Sydney si...,POLITICS,SOCIAL,0
6409,6409,"Glad to hear the #sydneysiege is over, but sad...",POLITICS,SOCIAL,0


In [ ]:
all_testing_health = all_testing[all_testing["predicted_domain"] == 'HEALTH']
all_testing_health.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/FurtherTesting_Datasets/further_testing_health.csv")
all_testing_health

,id,text,expected_domain,predicted_domain,label
1693,1693,The Ministry of Health in Bermuda has confirme...,HEALTH,HEALTH,1
1697,1697,Delhi: Eight nurses test positive for Covid-19...,HEALTH,HEALTH,1
1698,1698,"When the coronavirus outbreak began, the Cente...",HEALTH,HEALTH,0
1699,1699,A Pearl River County man is recovering at home...,HEALTH,HEALTH,1
1700,1700,General view of the campus of Purdue Universit...,HEALTH,HEALTH,0
...,...,...,...,...,...
6419,6419,Sydney siege ends as police storm Lindt Cafe a...,POLITICS,HEALTH,0
6420,6420,"Breaking News: #SydneySiege is over, according...",POLITICS,HEALTH,0
6421,6421,Watch gunfire erupt and hostages flee chocolat...,POLITICS,HEALTH,0
6422,6422,Authorities have confirmed that #sydneysiege i...,POLITICS,HEALTH,0


In [ ]:
all_testing_science = all_testing[all_testing["predicted_domain"] == 'SCIENCE']
all_testing_science.to_csv("/content/drive/MyDrive/CSI4900/Outputs/Overall_Testing_Dataset_Results/FurtherTesting_Datasets/further_testing_science.csv")
all_testing_science

,id,text,expected_domain,predicted_domain,label
1775,1775,Can someone tell these fucking idiots in Perth...,HEALTH,SCIENCE,0
2143,2143,Chinese people lining up in a crowd w/o a mask...,HEALTH,SCIENCE,0
2183,2183,"Dr. Jane Orient: The worst, possibly existenti...",HEALTH,SCIENCE,0
2231,2231,In todays interview recorded on the 13 th Febr...,HEALTH,SCIENCE,0
2856,2856,Certainly a topic which none would like to rea...,HEALTH,SCIENCE,1
...,...,...,...,...,...
6348,6348,All my thoughts &amp; prayers are with the peo...,POLITICS,SCIENCE,0
6349,6349,Name of #sydneysiege gunman has been revealed-...,POLITICS,SCIENCE,0
6357,6357,Thoughts go out to the people in Sydney today!...,POLITICS,SCIENCE,0
6390,6390,BREAKING: Police say Sydney cafe hostage situa...,POLITICS,SCIENCE,0


In [ ]:
len(all_testing_science) + len(all_testing_social) + len(all_testing_crime) + len(all_testing_health) + len(all_testing_politics)

52365

In [2]:
import pandas as pd
import numpy as np

In [3]:
test_data = pd.read_csv('/content/drive/MyDrive/CSI4900/Datasets/ProcessedTrainingDatasets/MergedDatasets/LiarRemoved/test_df.csv')
test_data

,Unnamed: 0.1,Unnamed: 0,text,label,metadata,domain
0,64738,64738,"JINHUA, China (Reuters) - There’s no masking t...",0,"[{'article': None, 'author': None, 'date': 'Ma...",POLITICS
1,27677,27677,Conan O'Brien trains to be Dwayne Johnson's Ra...,0,"[{'article': None, 'author': None, 'date': Non...",SOCIAL
2,44029,44029,NEWARK — Almost as soon as Chris Christie w...,0,"[{'article': None, 'author': 'Kate Zernike and...",POLITICS
3,39079,39079,A dollar to be made and all I got was a quarte...,1,"[{'article': None, 'author': 'Spifflove', 'dat...",POLITICS
4,4751,4751,A U.S. resident who lately returned from a tri...,0,"[{'article': None, 'author': None, 'date': Non...",HEALTH
...,...,...,...,...,...,...
10021,82459,82459,A group of white nationalists and skinheads wh...,1,"[{'article': None, 'author': None, 'date': 'Ju...",POLITICS
10022,76058,76058,WASHINGTON (Reuters) - The U.S. Transportation...,0,"[{'article': None, 'author': None, 'date': 'Se...",POLITICS
10023,34585,34585,Humiliated Hillary Tries To Hide What Camera C...,1,"[{'article': None, 'author': 'Amanda Shea', 'd...",POLITICS
10024,70792,70792,WASHINGTON (Reuters) - The U.S. State Departme...,0,"[{'article': None, 'author': None, 'date': 'No...",POLITICS


In [5]:
test_data['domain'].value_counts()

domain
POLITICS    6570
SOCIAL      2214
HEALTH      1071
SCIENCE       90
CRIME         81
Name: count, dtype: int64

In [6]:
test_data['label'].value_counts()

label
0    5484
1    4542
Name: count, dtype: int64

In [7]:
test1 = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/testing_all1.csv")
test1

,Unnamed: 0,text,label,metadata,domain
0,0,Does This Video Show a Thwarted Mothers Day Ro...,0,"[{'article_category_phase1': 'Crime', 'article...",CRIME
1,1,Should You Call #77 or 112 to Contact the Poli...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME
2,2,Should You Call #77 or 112 to Contact the Poli...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME
3,3,Was a Child Named Chloe Jones Abducted by a Cr...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME
4,4,Was a Child Named Chloe Jones Abducted by a Cr...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME
...,...,...,...,...,...
23830,23830,HAVANA (Reuters) - The United States decision...,0,NaN,SOCIAL
23831,23831,WASHINGTON (Reuters) - The U.S. led coalition ...,0,NaN,SOCIAL
23832,23832,KABUL (Reuters) - Islamic State claimed respon...,0,NaN,SOCIAL
23833,23833,WASHINGTON (Reuters) - The U.S. Justice Depart...,0,NaN,SOCIAL


In [8]:
test2 = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/testing_all2.csv")
test2

,Unnamed: 0,text,label,metadata,domain
0,23835,COX S BAZAR (Reuters) - Grieving Rohingya Musl...,0,NaN,SOCIAL
1,23836,DAR ES SALAAM (Reuters) - Tanzania shut down a...,0,NaN,SOCIAL
2,23837,MADRID (Reuters) - Catalonia s High Court on F...,0,NaN,SOCIAL
3,23838,"ERBIL, Iraq (Reuters) - An ban on internationa...",0,NaN,SOCIAL
4,23839,TALLINN (Reuters) - French President Emmanuel ...,0,NaN,SOCIAL
...,...,...,...,...,...
23831,47666,Says the large trade deficit with Japan stems ...,1,"[{'author': 'Donald Trump', 'factcheck_analysi...",POLITICS
23832,47667,"""Tens of thousands"" of people leave New York e...",1,"[{'author': 'Donald Trump Jr.', 'factcheck_ana...",POLITICS
23833,47668,"""I have fought for our shared values without b...",1,"[{'author': 'Chris Abele', 'factcheck_analysis...",POLITICS
23834,47669,"""Germany halts all Covid-19 vaccines, says the...",1,"[{'author': 'Bloggers', 'factcheck_analysis_li...",POLITICS


In [9]:
test3 = pd.read_csv("/content/drive/MyDrive/CSI4900/Datasets/ProcessedTestingDatasets/politics_pheme.csv")
test3

,Unnamed: 0.1,Unnamed: 0,text,label,author,metadata,domain
0,0,0,"Breaking: At least 10 dead, 5 injured after tO...",0,H_E_Samuel,{'account_date': 'Tue Oct 04 09:36:17 +0000 20...,POLITICS
1,1,1,France: 10 people dead after shooting at HQ of...,0,euronews,{'account_date': 'Wed Mar 18 12:57:11 +0000 20...,POLITICS
2,2,2,Ten killed in shooting at headquarters of Fren...,0,Channel4News,{'account_date': 'Mon Apr 28 14:29:37 +0000 20...,POLITICS
3,3,3,BREAKING: 10 dead in shooting at headquarters ...,0,AlArabiya_Eng,{'account_date': 'Sat Feb 28 08:31:32 +0000 20...,POLITICS
4,4,4,Reuters: 10 people shot dead at headquarters o...,0,SkyNewsBreak,{'account_date': 'Wed Nov 04 11:25:45 +0000 20...,POLITICS
...,...,...,...,...,...,...,...
6419,6419,6419,Sydney siege ends as police storm Lindt Cafe a...,0,guardian,{'account_date': 'Thu Nov 05 23:49:19 +0000 20...,POLITICS
6420,6420,6420,"Breaking News: #SydneySiege is over, according...",0,FoxNews,{'account_date': 'Sat Mar 17 19:01:26 +0000 20...,POLITICS
6421,6421,6421,Watch gunfire erupt and hostages flee chocolat...,0,NBCNews,{'account_date': 'Tue Mar 18 23:19:17 +0000 20...,POLITICS
6422,6422,6422,Authorities have confirmed that #sydneysiege i...,0,YahooNewsUK,{'account_date': 'Fri Feb 13 15:44:54 +0000 20...,POLITICS


In [11]:
test_full = pd.concat([test1, test2, test3])
test_full = test_full.drop_duplicates(subset=['text'])
display(test_full)

,Unnamed: 0,text,label,metadata,domain,Unnamed: 0.1,author
0,0,Does This Video Show a Thwarted Mothers Day Ro...,0,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN,NaN
1,1,Should You Call #77 or 112 to Contact the Poli...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN,NaN
3,3,Was a Child Named Chloe Jones Abducted by a Cr...,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN,NaN
5,5,Colored Stickers Target Homes of Dog Owners?,1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN,NaN
15,15,"Crime in Sweden, Part III: Does Sweden Have No...",1,"[{'article_category_phase1': 'Crime', 'article...",CRIME,NaN,NaN
...,...,...,...,...,...,...,...
6419,6419,Sydney siege ends as police storm Lindt Cafe a...,0,{'account_date': 'Thu Nov 05 23:49:19 +0000 20...,POLITICS,6419.0,guardian
6420,6420,"Breaking News: #SydneySiege is over, according...",0,{'account_date': 'Sat Mar 17 19:01:26 +0000 20...,POLITICS,6420.0,FoxNews
6421,6421,Watch gunfire erupt and hostages flee chocolat...,0,{'account_date': 'Tue Mar 18 23:19:17 +0000 20...,POLITICS,6421.0,NBCNews
6422,6422,Authorities have confirmed that #sydneysiege i...,0,{'account_date': 'Fri Feb 13 15:44:54 +0000 20...,POLITICS,6422.0,YahooNewsUK


In [14]:
test_full['label'].value_counts()

label
1    26579
0    25786
Name: count, dtype: int64